In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pandas as pd

In [2]:
df1 = spark.createDataFrame(pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': ['one', 'two', 'three', 'five']}))
df2 = spark.createDataFrame(pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': ['five', 'six', None, None]}))
df1.show()

df2.show()

+----+-----+
lkey|value|
+----+-----+
 foo| one|
 bar| two|
 baz|three|
 foo| five|
+----+-----+

+----+-----+
rkey|value|
+----+-----+
 foo| five|
 bar| six|
 baz| null|
 foo| null|
+----+-----+

In [3]:
from pyspark.sql import functions as F

df1.withColumnRenamed("value","value_x")\
    .join(df2.withColumnRenamed("value","value_y"),F.col("lkey")==F.col("rkey"),'left')\
    .withColumn("value_x", F.when(F.col("value_y").isNotNull(),F.col("value_y")).otherwise(F.col("value_x"))).show()


#+----+-------+----+-------+
#|lkey|value_x|rkey|value_y|
#+----+-------+----+-------+
#| bar|    six| bar|    six|
#| foo|   five| foo|   five|
#| foo|    one| foo|   null|
#| foo|   five| foo|   five|
#| foo|   five| foo|   null|
#| baz|  three| baz|   null|
#+----+-------+----+-------+

+----+-------+----+-------+
lkey|value_x|rkey|value_y|
+----+-------+----+-------+
 bar| six| bar| six|
 foo| five| foo| five|
 foo| one| foo| null|
 foo| five| foo| five|
 foo| five| foo| null|
 baz| three| baz| null|
+----+-------+----+-------+

In [4]:
  lkey value_x rkey value_y
0  foo    five  foo    five
1  foo     one  foo     NaN
2  bar     six  bar     six
3  baz   three  baz     NaN
4  foo    five  foo    five
5  foo    five  foo     NaN

In [5]:
list=[[95110]]

df=spark.createDataFrame(list,['code'])

df.show()


+-----+
 code|
+-----+
95110|
+-----+

In [6]:
list=[["1 2 3 4 5 6 7 8 9 10"]]

df=spark.createDataFrame(list,['x'])

In [7]:
df.select(*[(F.split("x",' ')[i]).alias(str(i)) for i in range(100)])

#== Physical Plan ==
#*(1) Project [split(x#200,  )[0] AS 0#1708, split(x#200,  )[1] AS 1#1709, split(x#200,  )[2] AS 2#1710, split(x#200,  )[3] AS 3#1711, split(x#200,  )[4] AS 4#1712, split(x#200,  )[5] AS 5#1713, split(x#200,  )[6] AS 6#1714, split(x#200,  )[7] AS 7#1715, split(x#200,  )[8] AS 8#1716, split(x#200,  )[9] AS 9#1717, split(x#200,  )[10] AS 10#1718, split(x#200,  )[11] AS 11#1719, split(x#200,  )[12] AS 12#1720, split(x#200,  )[13] AS 13#1721, split(x#200,  )[14] AS 14#1722, split(x#200,  )[15] AS 15#1723, split(x#200,  )[16] AS 16#1724, split(x#200,  )[17] AS 17#1725, split(x#200,  )[18] AS 18#1726, split(x#200,  )[19] AS 19#1727, split(x#200,  )[20] AS 20#1728, split(x#200,  )[21] AS 21#1729, split(x#200,  )[22] AS 22#1730, split(x#200,  )[23] AS 23#1731, ... 76 more fields]
#+- *(1) Scan ExistingRDD[x#200]

== Physical Plan ==
*(1) Project [split(x#200, )[0] AS 0#1708, split(x#200, )[1] AS 1#1709, split(x#200, )[2] AS 2#1710, split(x#200, )[3] AS 3#1711, split(x#200, )[4] AS 4#1712, split(x#200, )[5] AS 5#1713, split(x#200, )[6] AS 6#1714, split(x#200, )[7] AS 7#1715, split(x#200, )[8] AS 8#1716, split(x#200, )[9] AS 9#1717, split(x#200, )[10] AS 10#1718, split(x#200, )[11] AS 11#1719, split(x#200, )[12] AS 12#1720, split(x#200, )[13] AS 13#1721, split(x#200, )[14] AS 14#1722, split(x#200, )[15] AS 15#1723, split(x#200, )[16] AS 16#1724, split(x#200, )[17] AS 17#1725, split(x#200, )[18] AS 18#1726, split(x#200, )[19] AS 19#1727, split(x#200, )[20] AS 20#1728, split(x#200, )[21] AS 21#1729, split(x#200, )[22] AS 22#1730, split(x#200, )[23] AS 23#1731, ... 76 more fields]
+- *(1) Scan ExistingRDD[x#200]

In [8]:
from pyspark.sql import functions as F
df\
  .withColumn("x", F.split('x',' '))\
  .select(*[(F.col("x")[i]).alias(str(i)) for i in range(100)]).drop("x").explain()

#== Physical Plan ==
#*(1) Project [x#1908[0] AS 0#1910, x#1908[1] AS 1#1911, x#1908[2] AS 2#1912, x#1908[3] AS 3#1913, x#1908[4] AS 4#1914, x#1908[5] AS 5#1915, x#1908[6] AS 6#1916, x#1908[7] AS 7#1917, x#1908[8] AS 8#1918, x#1908[9] AS 9#1919, x#1908[10] AS 10#1920, x#1908[11] AS 11#1921, x#1908[12] AS 12#1922, x#1908[13] AS 13#1923, x#1908[14] AS 14#1924, x#1908[15] AS 15#1925, x#1908[16] AS 16#1926, x#1908[17] AS 17#1927, x#1908[18] AS 18#1928, x#1908[19] AS 19#1929, x#1908[20] AS 20#1930, x#1908[21] AS 21#1931, x#1908[22] AS 22#1932, x#1908[23] AS 23#1933, ... 76 more fields]
#+- *(1) Project [split(x#200,  ) AS x#1908]
#   +- *(1) Scan ExistingRDD[x#200]

== Physical Plan ==
*(1) Project [x#2210[0] AS 0#2212, x#2210[1] AS 1#2213, x#2210[2] AS 2#2214, x#2210[3] AS 3#2215, x#2210[4] AS 4#2216, x#2210[5] AS 5#2217, x#2210[6] AS 6#2218, x#2210[7] AS 7#2219, x#2210[8] AS 8#2220, x#2210[9] AS 9#2221, x#2210[10] AS 10#2222, x#2210[11] AS 11#2223, x#2210[12] AS 12#2224, x#2210[13] AS 13#2225, x#2210[14] AS 14#2226, x#2210[15] AS 15#2227, x#2210[16] AS 16#2228, x#2210[17] AS 17#2229, x#2210[18] AS 18#2230, x#2210[19] AS 19#2231, x#2210[20] AS 20#2232, x#2210[21] AS 21#2233, x#2210[22] AS 22#2234, x#2210[23] AS 23#2235, ... 76 more fields]
+- *(1) Project [split(x#200, ) AS x#2210]
 +- *(1) Scan ExistingRDD[x#200]

In [9]:
from pyspark.sql import functions as F
df.withColumn("contains_5digit", F.expr("""IF(length(code)==5,1,0)""")).show()

+-----+---------------+
 code|contains_5digit|
+-----+---------------+
95110| 1|
+-----+---------------+

In [10]:
df = spark.createDataFrame([[{"x": 30.0, "pool": 20.0, "helium": 10.0}, -5],
                           [{"x": 40.0, "pool": 30.0, "helium": 20.0}, 5]],["col1", "col2"])

df.show()

+--------------------+----+
 col1|col2|
+--------------------+----+
[pool -> 20.0, x ...| -5|
[pool -> 30.0, x ...| 5|
+--------------------+----+

In [12]:
+------------------------------+-----+
|col1                          |col2 |
+------------------------------+-----+
|"x: 1.0, y: 2.0, z: 3.0"      | 5.0 |
|"x: 4.0, y: 5.0, z: 6.0"      | 5.0 |
+------------------------------+-----+

In [13]:
display(df.withColumn("col1", F.col("col1").cast("string"))).show()

col1,col2
"[pool -> 20.0, x -> 30.0, helium -> 10.0]",-5
"[pool -> 30.0, x -> 40.0, helium -> 20.0]",5


In [14]:
df.withColumn("col1",F.concat(F.lit('"'),F.col("col1").cast("string"),F.lit('"'))).show(truncate=False)

#+------------------------------+----+
#|col1                          |col2|
#+------------------------------+----+
#|{pool:20.0,x:30.0,helium:10.0}|-5  |
#|{pool:30.0,x:40.0,helium:20.0}|5   |
#+------------------------------+----+

+-------------------------------------------+----+
col1 |col2|
+-------------------------------------------+----+
"[pool -> 20.0, x -> 30.0, helium -> 10.0]"|-5 |
"[pool -> 30.0, x -> 40.0, helium -> 20.0]"|5 |
+-------------------------------------------+----+

In [15]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': ['one', 'two', 'three', 'five']})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': ['five', 'six', nan, nan]})final_reference = clean_reference.withColumn("OutputItemNameByValue",\
                       f.when((f.col("OutputItemNameByValue") == " ")|                              
                             (f.col("PrimaryLookupAttributeValue") ==\
                              "TRIANA_DEFAULT"), f.col("OutputItemNameByValue"))\
                                             .otherwise(f.lit("Not Implemented")))

In [16]:
final_reference = clean_reference.withColumn("OutputItemNameByValue",F.expr("clean_reference.OutputItemNameByValue == " " or\   clean_reference.PrimaryLookupAttributeValue == 'TRIANA_DEFAULT', clean_reference.OutputItemNameByValue,Not Implemented"

In [17]:
df = spark.createDataFrame([(0, 'a', 1),
                                 (1, 'a', 1),
                                 (2, 'a', 1),
                                 (3, 'a', 1)],['day', 'user', 'raw_score'])
df.show()

+---+----+---------+
day|user|raw_score|
+---+----+---------+
 0| a| 1|
 1| a| 1|
 2| a| 1|
 3| a| 1|
+---+----+---------+

In [18]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType


@pandas_udf(df.withColumn("raw_score", F.lit(1.2456)).schema, PandasUDFType.GROUPED_MAP)
def grouped_map(df):
     for i in range(1,len(df)):
          df.loc[i,'raw_score']=(df.loc[i-1,'raw_score'] * 0.9)+1   

     return df
df\
  .groupby("user").apply(grouped_map).show()

#+---+----+---------+
#|day|user|raw_score|
#+---+----+---------+
#|  0|   a|      1.0|
#|  1|   a|      1.9|
#|  2|   a|     2.71|
#|  3|   a|    3.439|
#+---+----+---------+

+---+----+---------+
day|user|raw_score|
+---+----+---------+
 0| a| 1.0|
 1| a| 1.9|
 2| a| 2.71|
 3| a| 3.439|
+---+----+---------+

In [19]:
df.withColumn("raw_score", F.lit(1.2456)).schema

Out[4]: StructType(List(StructField(day,LongType,true),StructField(user,StringType,true),StructField(raw_score,DoubleType,false)))

In [20]:
list=[[4  ,8  ,1  ,1 ,2  ,2  ,8   ,3  ,5],
      [7  ,5  ,6  ,1 ,6  ,2  ,3   ,3  ,1],
      [7  ,3  ,5  ,1 ,9  ,2  ,4   ,3  ,7]]

df=spark.createDataFrame(list,['x','y','z','a','ad','b','bd','c','cd'])

df.show()

+---+---+---+---+---+---+---+---+---+
 x| y| z| a| ad| b| bd| c| cd|
+---+---+---+---+---+---+---+---+---+
 4| 8| 1| 1| 2| 2| 8| 3| 5|
 7| 5| 6| 1| 6| 2| 3| 3| 1|
 7| 3| 5| 1| 9| 2| 4| 3| 7|
+---+---+---+---+---+---+---+---+---+

In [21]:
df.withColumn("zip", F.arrays_zip(F.array('a','b','c'),F.array('ad','bd','cd')))\
  .withColumn("min", F.array_min(F.array("ad","bd","cd")))\
  .printSchema()

root
-- x: long (nullable = true)
-- y: long (nullable = true)
-- z: long (nullable = true)
-- a: long (nullable = true)
-- ad: long (nullable = true)
-- b: long (nullable = true)
-- bd: long (nullable = true)
-- c: long (nullable = true)
-- cd: long (nullable = true)
-- zip: array (nullable = false)
 |-- element: struct (containsNull = false)
 | |-- 0: long (nullable = true)
 | |-- 1: long (nullable = true)
-- min: long (nullable = true)

In [22]:
df.withColumn("XX", F.array('a','b','c')).withColumn("XD", F.array('ad','bd','cd'))\
  .withColumn("min", F.array_min("XD")).withColumn("zip", F.arrays_zip("XX","XD"))\
  .withColumn("id", F.expr("""filter(zip,x-> x.XD=min)"""))\
  .select("x","y","z", (F.col("id.XX")[0]).alias("id"))\
  .show(truncate=False)

#+---+---+---+---+
#|x  |y  |z  |id |
#+---+---+---+---+
#|4  |8  |1  |1  |
#|7  |5  |6  |3  |
#|7  |3  |5  |2  |
#+---+---+---+---+

+---+---+---+---+
x |y |z |id |
+---+---+---+---+
4 |8 |1 |1 |
7 |5 |6 |3 |
7 |3 |5 |2 |
+---+---+---+---+

In [23]:
from pyspark.sql import functions as F

df.withColumn("zip", F.arrays_zip(F.array('a','b','c'),F.array('ad','bd','cd')))\
  .withColumn("id", F.expr("""filter(zip,x-> x.`1`=array_min(array(ad,bd,cd)))"""))\
  .select("x","y","z", (F.col("id.0")[0]).alias("id")).show()

#+---+---+---+---+
#|  x|  y|  z| id|
#+---+---+---+---+
#|  4|  8|  1|  1|
#|  7|  5|  6|  3|
#|  7|  3|  5|  2|
#+---+---+---+---+

+---+---+---+---+
 x| y| z| id|
+---+---+---+---+
 4| 8| 1| 1|
 7| 5| 6| 3|
 7| 3| 5| 2|
+---+---+---+---+

In [24]:
list=[['a', 2, 10],
      ['a' ,1 ,14],
      ['a' ,4 ,5],
      ['b' ,1 ,4]]

df=spark.createDataFrame(list,['user','day','amount'])

df.show()

+----+---+------+
user|day|amount|
+----+---+------+
 a| 2| 10|
 a| 1| 14|
 a| 4| 5|
 b| 1| 4|
+----+---+------+

In [25]:
a=df.agg(F.min("day"),F.max("day")).collect()[0]

a[0]




Out[28]: 1

In [26]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy(F.lit(0))

df.withColumn("boundaries", F.sequence(F.min("day").over(w),F.max("day").over(w),F.lit(1)))\
  .groupBy("user").agg(F.collect_list("day").alias('day'),F.collect_list("amount").alias('amount')\
   ,F.first("boundaries").alias("boundaries")).withColumn("boundaries", F.array_except("boundaries","day"))\
  .withColumn("day",F.flatten(F.array("day","boundaries"))).drop("boundaries")\
  .withColumn("zip", F.explode(F.arrays_zip("day","amount")))\
  .select("user","zip.day", F.when(F.col("zip.amount").isNull(),\
                                   F.lit(0)).otherwise(F.col("zip.amount")).alias("amount")).show()
#+----+---+------+
#|user|day|amount|
#+----+---+------+
#|   a|  2|    10|
#|   a|  1|    14|
#|   a|  4|     5|
#|   a|  3|     0|
#|   b|  1|     4|
#|   b|  2|     0|
#|   b|  3|     0|
#|   b|  4|     0|
#+----+---+------+


+----+---+------+
user|day|amount|
+----+---+------+
 b| 1| 4|
 b| 2| 0|
 b| 3| 0|
 b| 4| 0|
 a| 2| 10|
 a| 1| 14|
 a| 4| 5|
 a| 3| 0|
+----+---+------+

In [27]:
#+----+---+------+
#|user|day|amount|
#+----+---+------+
#|   a|  2|    10|
#|   a|  1|    14|
#|   a|  4|     5|
#|   a|  3|     0|
#|   b|  1|     4|
#|   b|  2|     0|
#|   b|  3|     0|
#|   b|  4|     0|
#+----+---+------+